In [1]:
import numpy as n
%load_ext autoreload
%autoreload 2

In [2]:
# import jax.tools.colab_tpu
# jax.tools.colab_tpu.setup_tpu()

In [3]:
from model import train_model
from constraint_problems import get_problem_from_cnf
from random_walk import moser_walk_sampler
import jax

In [49]:
from pysat.formula import CNF
import glob
instances = [get_problem_from_cnf(CNF(from_file=f)) for f in glob.glob('../Data/uf20-91/*.cnf')]
train_instances, test_instances = instances[:950],instances[950:]

In [56]:
train_model(1000,0.1, train_instances, test_instances)

KeyboardInterrupt: 

In [9]:
from pysat.formula import CNF
planning_problem = get_problem_from_cnf(CNF(from_file="../Data/blocksworld/medium.cnf"))

In [14]:
network, params = train_model(1000,0.1,[planning_problem], [planning_problem], num_steps=50)

step %r loss train %r test %r 0 0.6930736303329468
step %r loss train %r test %r 10 0.6928116679191589
step %r loss train %r test %r 20 0.6922646164894104
step %r loss train %r test %r 30 0.6913454532623291
step %r loss train %r test %r 40 0.6905084848403931


In [12]:
prdictions = network.apply(params, planning_problem.graph)
weights = jax.nn.softmax(prdictions)[: planning_problem.params[0], 1]
trajectory, energies = moser_walk_sampler(weights, planning_problem, 1000)

In [13]:
energies

DeviceArray([201, 200, 200, 201, 201, 201, 201, 199, 199, 198, 194, 191,
             191, 196, 196, 197, 197, 196, 193, 193, 187, 186, 183, 211,
             199, 195, 190, 191, 191, 191, 188, 185, 184, 189, 182, 183,
             184, 196, 185, 182, 185, 182, 182, 183, 184, 185, 182, 186,
             187, 186, 186, 186, 186, 186, 190, 188, 188, 187, 182, 182,
             192, 186, 186, 187, 186, 186, 183, 182, 182, 184, 184, 182,
             186, 182, 184, 184, 182, 182, 182, 191, 185, 184, 189, 185,
             184, 184, 189, 189, 185, 182, 182, 182, 182, 186, 184, 184,
             202, 193, 188, 186, 185, 184, 185, 182, 182, 182, 183, 184,
             215, 204, 197, 187, 187, 187, 187, 187, 185, 182, 183, 182,
             187, 186, 187, 185, 185, 185, 185, 184, 185, 182, 182, 182,
             180, 185, 182, 182, 181, 180, 180, 185, 185, 182, 182, 182,
             182, 184, 182, 182, 182, 185, 183, 182, 182, 182, 182, 182,
             182, 185, 185, 182, 182, 183, 183, 183